In [1]:
from qiskit_aer.primitives import Estimator
from qiskit_aer import Aer

import scipy.linalg as ln

from Ring import *

#### Retriving the fake Backend

In [2]:
from qiskit_ibm_runtime.fake_provider import FakeBrisbane

fake_backend = FakeBrisbane()
fake_backend = Aer.get_backend("aer_simulator") # EXACT -- ESTIMATOR

In [3]:
# Fixing the qubits to be used on the QPU 
# layout = [37, 38, 39, 40 ,41, 53, 60, 59, 58, 57, 56, 52]

# Fanne uno per ognuno degli hardware in modo da utilizzare
# sempre il migliore possibile [TODO]

# Puoi anche investigare il plug_in "mapomatic" che vuole
# automatizzare questo step. [TODO]
# https://github.com/qiskit-community/mapomatic

#### Setting the number of sites of the ring ($\equiv$ number of qubit used)

In [4]:
# Ring sites
n_spin = 12

# Creation of the system Hamiltonian as SparsePauliOp
H = build_hamiltonian(n_spin)

#### Generating the annealing circuit

In [5]:
# Number of trotter step -- each trotter step is 15-CNOT deep
n_tstep = 4

# Creation of the annealing circuit
qc = trotterized_annealing(n_spin, n_tstep)

#### Transpiling the circuit to match the backend

In [6]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
pm = generate_preset_pass_manager(optimization_level=2, backend=fake_backend)

In [7]:
# Transpile the circuit to an "Instruction Set Architecture" (ISA) circuit.
# Note: the transpiler automatically adds "ancilla" qubits to make the transpiled
# circuit match the size of the FakeSherbrooke backend.
isa_circ = pm.run(qc)

# circ.draw()

#### Execution of the energy estimation throught a Krylov-like Subspace Expansion.
##### Call $|\psi\rangle$ the state obtained by the execution of the annealing circuit, then we build the space $\{\mathrm{I}|\psi\rangle, H|\psi\rangle, H^2|\psi\rangle\}$ and solve the generilized eigenvalue problem $$ \mathcal{H}\nu = \varepsilon\mathcal{S}\nu $$ where $\mathcal{H}_{i,j} = \langle\psi|H^{i+j+1}|\psi\rangle$ and $\mathcal{S}_{i,j} = \langle\psi|H^{i+j}|\psi\rangle$

In [8]:
# Setting the maximum power of the Hamiltonian to use
max_pow = 1

# Allocating the estiamtor to use for the Subpspace Expansion
estimator = Estimator(mode=fake_backend)

# Retriving an energy estimation throught digitalized annealing + subspace expansion
estimate = energy_estimate_SbE(isa_circ, H, estimator, max_pow)

#### Computing the exact ground state of the Hamiltonian

In [9]:
# Computing a numerical value of the system Hamiltonian [TBD]
H_num = evaluate_hamiltonian(H)

# Numerical diagonalization of the Hamiltonian [TBD]
e, V = ln.eigh(np.real(H_num), subset_by_index=[0, 0])

# Recuperiamo l'indice del groung state
id_gs = np.argmin(e)

# Recuperiamo il ground state e la relativa energia
gs     = V[:,id_gs]
energy = e[id_gs]

#### Comparison of the estimate from the Subspace Expansion with the exact (numerical) value of the ground state energy

In [10]:
err = abs(energy - estimate) / abs(energy)
print([energy, estimate, err])

[-5.387390917445205, array([-5.25407175]), array([0.02474652])]
